# Trend Screener

In [426]:
pip install arch scipy yfinance pandas numpy QuantLib-Python


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [427]:
import datetime as dt
import pandas as pd
import yfinance as yf
import numpy as np
from arch import arch_model
from scipy.stats import norm
import QuantLib as ql
from datetime import datetime
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor, as_completed

In [428]:
tickers = ['CLOV', 'ASTS', 'WGS', 'NSSC', 'AAPL']

risk_free_rate = 0.05

end_date = '2024-09-27'

start_date_1d = (pd.to_datetime(end_date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
start_date_3m = (pd.to_datetime(end_date) - pd.DateOffset(months=3)).strftime('%Y-%m-%d')
start_date_12m = (pd.to_datetime(end_date) - pd.DateOffset(months=12)).strftime('%Y-%m-%d')


stock_price = yf.download(tickers, start = start_date_1d, end = end_date)

[*********************100%%**********************]  5 of 5 completed


In [429]:
def process_expiration(tk, exp_td_str):
    options = tk.option_chain(exp_td_str)
    
    calls = options.calls
    puts = options.puts
    
    calls['optionType'] = 'call'
    puts['optionType'] = 'put'
    
    exp_data = pd.concat(objs=[calls, puts], ignore_index=True)

    
    return exp_data


def fetch_option_price(ticker):
    tk = yf.Ticker(ticker)
    expirations = tk.options

    data = pd.DataFrame()

    for exp_td_str in expirations:
        exp_data = process_expiration(tk, exp_td_str)
        data = pd.concat(objs=[data, exp_data], ignore_index=True)

    data['close'] = tk.history(period='1d')['Close'].iloc[-1]

    return data

ticker = "AAPL"
option_data = fetch_option_price(ticker)

In [ ]:

def price_heston_option(
    spot_price, 
    strike_price, 
    risk_free_rate, 
    dividend_yield, 
    initial_volatility, 
    expiry, 
    kappa, 
    theta, 
    sigma, 
    rho, 
    option_type='call'
):
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    rate_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), ql.Actual360())
    )
    dividend_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), ql.Actual360())
    )

    heston_process = ql.HestonProcess(
        rate_handle,  # Risk-free rate
        dividend_handle,  # Dividend yield
        spot_handle,  # Initial stock price
        initial_volatility ** 2,  # Initial variance (square of initial volatility)
        kappa,  # Mean reversion speed
        theta,  # Long-term variance
        sigma,  # Volatility of volatility
        rho  # Correlation
    )
    heston_model = ql.HestonModel(heston_process)

    expiry_date = ql.Date().todaysDate() + ql.Period(int(expiry), ql.Days)
    exercise = ql.EuropeanExercise(expiry_date)
    payoff_type = ql.Option.Call if option_type.lower() == 'call' else ql.Option.Put
    payoff = ql.PlainVanillaPayoff(payoff_type, strike_price)
    european_option = ql.VanillaOption(payoff, exercise)

    engine = ql.AnalyticHestonEngine(heston_model)
    european_option.setPricingEngine(engine)

    option_price = european_option.NPV()
    return option_price

In [ ]:
def fetch_and_calculate_garch_volatility(ticker, start_date, end_date, p=1, q=1):
    """
    Fetch historical stock price data, calculate daily returns, and compute GARCH volatility.
    
    Parameters:
    - ticker: The stock ticker symbol (e.g., 'AAPL').
    - start_date: The start date for fetching data (format 'YYYY-MM-DD').
    - end_date: The end date for fetching data (format 'YYYY-MM-DD').
    - p: The order of the GARCH term.
    - q: The order of the ARCH term.
    
    Returns:
    - A pandas Series of estimated conditional volatilities.
    """
    
    # Fetch historical stock price data
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Calculate daily returns
    returns = stock_data['Adj Close'].pct_change().dropna()

    # Fit GARCH model and calculate volatility
    model = arch_model(returns, vol='Garch', p=p, q=q, rescale=True)
    model_fit = model.fit(disp="off")
    daily_volatility = model_fit.conditional_volatility

    # Convert to annual volatility (assuming approximately 252 trading days)
    annual_volatility = daily_volatility * np.sqrt(252) / 10

    return annual_volatility

In [ ]:

def calc_expiry_day(ticker, option_symbol):
    option_symbol_clean = option_symbol.replace(ticker, '')
    expiration_code = option_symbol_clean[:6]

    year = '20' + expiration_code[:2]
    month = expiration_code[2:4]
    day = expiration_code[4:6]

    expiration_date = f"{year}-{month}-{day}"
    expiry = (pd.to_datetime(expiration_date).tz_localize('UTC') - pd.to_datetime(end_date).tz_localize('UTC')).days

    return expiry

def screen_option(ticker):
    try:
        volatilities = fetch_and_calculate_garch_volatility(ticker, start_date_3m, end_date, p=1, q=1)
        long_term_volatilities = fetch_and_calculate_garch_volatility(ticker, start_date_12m, end_date, p=1, q=1)
        vol_of_vol = volatilities.std()
        option_prices = fetch_option_price(ticker)

        option_df = pd.DataFrame(option_prices)

        for index, row in option_df.iterrows():
            expiry_day = calc_expiry_day(ticker, row['contractSymbol'])

            price_theory = price_heston_option(
                spot_price=float(row['close']),
                strike_price=float(row['strike']),
                risk_free_rate=risk_free_rate,
                dividend_yield=0.0,
                initial_volatility=volatilities.iloc[-1],
                expiry=expiry_day,
                kappa=1.0,
                theta=long_term_volatilities.iloc[-1],
                sigma=vol_of_vol,
                rho=-0.5,
                option_type=row['optionType']
            )

            option_df.at[index, 'priceTheory'] = price_theory
            option_df.at[index, 'garchVol'] = volatilities.iloc[-1]


        return option_df
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return ticker, pd.DataFrame()

def screen_all_options(tickers):
    all_options_df = pd.DataFrame()

    for ticker in tickers:
        print(ticker)
        options_df = screen_option(ticker)
        options_df['ticker'] = ticker
        if not options_df.empty:
            all_options_df = pd.concat([all_options_df, options_df], ignore_index=True)

    return all_options_df

options_data = screen_all_options(tickers)

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

CLOV



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ASTS


...

WGS



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

NSSC



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

AAPL


In [ ]:
for index, row in options_data.iterrows():
  signal = None
  reward = 0

  if row['priceTheory'] * 1.5 <= row['bid'] and \
      row['priceTheory'] >= 0.1 and row['bid'] >=0.15 :
    signal = "SELL"
    reward = (row['bid'] - row['priceTheory']) / row['priceTheory']
  elif row['ask'] * 1.5 <= row['priceTheory'] and \
      row['priceTheory'] > 0.1 and row['ask'] >= 0.15:
    signal = "BUY"
    reward = (row['priceTheory'] - row['ask']) / row['ask']

  options_data.loc[index, 'signal'] = signal
  options_data.loc[index, 'reward'] = reward

summary_df = pd.DataFrame(options_data)
summary_df.to_csv('options_chain.csv')

print(summary_df)

           contractSymbol             lastTradeDate  strike  lastPrice    bid  \
0     CLOV241004C00001000 2024-09-20 15:19:34+00:00     1.0       1.99   1.38   
1     CLOV241004C00002000 2024-09-27 18:38:08+00:00     2.0       0.80   0.79   
2     CLOV241004C00002500 2024-09-27 19:04:52+00:00     2.5       0.34   0.32   
3     CLOV241004C00003000 2024-09-27 19:59:55+00:00     3.0       0.05   0.05   
4     CLOV241004C00003500 2024-09-27 19:48:48+00:00     3.5       0.01   0.01   
...                   ...                       ...     ...        ...    ...   
2636  AAPL270115P00220000 2024-09-26 15:29:32+00:00   220.0      23.80  23.00   
2637  AAPL270115P00230000 2024-09-26 18:45:28+00:00   230.0      27.95  27.30   
2638  AAPL270115P00240000 2024-09-24 18:12:30+00:00   240.0      33.00  30.80   
2639  AAPL270115P00250000 2024-09-26 19:21:47+00:00   250.0      38.02  37.15   
2640  AAPL270115P00260000 2024-09-27 15:04:50+00:00   260.0      43.50  42.95   

        ask  change  percen